In [1]:
import re
from itertools import combinations
from itertools import permutations
from collections import Counter
import pandas as pd

Teksti tükeldamise [funktsioon](https://github.com/PacktPublishing/Python-Machine-Learning-Cookbook/blob/master/Chapter06/chunking.py) `splitter`, mille sisendiks on tekst ja soovitud tekstiosade pikkus sõnade arvuna. Tekst tükeldatakse tühikute kohalt.

In [2]:
# Split a text into chunks 
def splitter(data, num_words):
    words = data.split(' ')
    output = []

    cur_count = 0
    cur_words = []
    for word in words:
        cur_words.append(word)
        cur_count += 1
        if cur_count == num_words:
            output.append(' '.join(cur_words))
            cur_words = []
            cur_count = 0

    output.append(' '.join(cur_words) )

    return output 

Loo failist asenduste leksikon. Algfailis olevaid isikunimed ühtlustatakse. Parandatakse mõned vead ning teisendatakse hüüdnimed ja eri nimekujud ühtsele kujule (Krõet --> Krõõt jne), kuna tegemist on sama isikuga ja oleme huvitatud tegelaskujude paaridest.

In [3]:
name_replacements = {}
with open("name-replacements.txt", 'r', encoding='utf-8') as f:
    for line in f:
       (key, val) = line.split(";")
       name_replacements[key.strip()] = val.strip()

Algfailis tehakse nimede asendused ning väljund kirjutatakse uude faili.

In [4]:
with open("Anton_Hansen_Tammsaare_Tode_ja_oigus_I.utf8", 'r', encoding="utf-8") as f:
    content = f.read()
    content = re.sub(r'\b' + '|'.join(name_replacements.keys()) + r'\b', lambda m: name_replacements[m.group(0)], content)

with open("Anton_Hansen_Tammsaare_Tode_ja_oigus_I_asendustega.utf8", "w", encoding = "utf-8") as outp:
    outp.write(content)

Loetakse sisse lemmatiseeritud fail, lemmad on moodustatud asendustega failist. Lemmatiseerimise etapp on koodis praegu vahele jäetud.

In [5]:
with open("Anton_Hansen_Tammsaare_Tode_ja_oigus_I_lemmad.txt", 'r', encoding="utf-8") as f:
    content = f.read()

Teksti tükeldamine 200-sõnalisteks juppideks.

In [6]:
text_chunks = splitter(content, 200)

Kogu tekstist moodustatakse kaheliikmelised kombinatsioonid.

In [7]:
combs = []
for chunk in text_chunks:
    combs.append(list(combinations(chunk.split(" "), 2)))

Tekstist võetakse välja ainult need kombinatsioonid, mille mõlema liikmes on mõni suurtäht. Kuna sisendiks oli lemmatiseeritud ja teisendatud fail, siis lausealgulised suurtähed jmt on muudetud väikesteks, seega on potentsiaalselt nimed ainult need, kus esineb suurtäht. Samuti jäetkase välja kombinatsioonid, kus mõlemad liikmed on samad ning kõik kaksikud sorteeritakse enne lõplikku listi lisamist tähestikuliselt.

In [8]:
unique_combs_in_chunks = []
for i in combs:
    for j in i:
        if any(x.isupper() for x in j[0]) and any(x.isupper() for x in j[1]) and j[0] != j[1]:
            unique_combs_in_chunks.append(tuple(sorted(j)))

Kuna ka lemmatiseeritud fail pole täiesti puhas, siis oleks vaja kombinatsioone veel natuke puhastada. Välja visatakse kõik kohanimed, kuna oleme huvitatud ainult isikunimede paaridest. Lisaks visatakse välja veel mõned valesti nimeks märgendatud sõnad. Selleks on eelnevalt tekitatud fail loendiga ebasoovitavatest sõnadest.

In [14]:
exclusion = []
with open("excluded-names.txt", 'r', encoding='utf-8') as f:
    for line in f:
        exclusion.append(line.strip())
        
unique_combs_in_chunks = [e for e in unique_combs_in_chunks if e[0] not in exclusion and e[1] not in exclusion]

Nimekombinatsioonidest moodustatakse sagedusloend.

In [15]:
combination_freqs = Counter(unique_combs_in_chunks)

Nimepaaride sagedustest moodustatakse tabel.

In [29]:
name_df = pd.DataFrame.from_dict(combination_freqs, orient='index').reset_index()
name_df = name_df.rename(columns={'index':'pair', 0:'Count'})
name_df['pair'] = name_df['pair'].apply(','.join)
name_df.head()

,pair,Count
0,"Maasik,Mari",4
1,"Eedi,Madis",1
2,"Madis,Nonäh",5
3,"Karja-Eedi,Madis",2
4,"Madis,Mart",1


Nimepaaride sagedustega tabelist tekitatakse tabel, kus mõlemad nimed ning sagedus kuuluvad eraldi veergudesse. Paaridega veerg eemaldatakse ning veergude järjekorda muudetakse.

In [30]:
name_df['Source'], name_df['Target'] = zip(*name_df['pair'].map(lambda x: x.split(',')))
name_df = name_df.drop('pair', axis=1)
name_df = name_df.reindex_axis(['Source', 'Target', 'Count'], axis=1)
name_df.head()

,Source,Target,Count
0,Maasik,Mari,4
1,Eedi,Madis,1
2,Madis,Nonäh,5
3,Karja-Eedi,Madis,2
4,Madis,Mart,1


Lõpuks võetakse välja ainult paarid, mille esinemissagedus on kümnest suurem. Väljund kirjutatakse eraldi faili.

In [32]:
name_df = name_df.query('Count > 9')
name_df.to_csv("name_pairs.csv", sep = ";", index=False, encoding = "utf-8")